<a href="https://www.kaggle.com/code/lukalafaye/hub-eau-dataset?scriptVersionId=211039430" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [102]:
import requests
import pandas as pd
from io import StringIO

def fetch_all_data(url, output_file, params, date_key_api_min, date_key_api_max, date_key, desc=False, json=False):
    size = params.get('size', 10)
    done = True

    # Make the initial request
    response = requests.get(url, params)
    print("Response obtained.")
    if response.status_code not in [200, 206]:
        done = False
        print(f"Error: {response.status_code} - {response.text}")
        return 

    if json:
        data = response.json()
        df = pd.json_normalize(data['data'])
    else:
        content = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(content), sep=";")

    df.to_csv(output_file, index=False, sep=";")

    print("before while")
    while True:
        stop = len(df)
        if stop == 0 or stop < size:
            done = True
            break

        last_date = df[date_key].iloc[-1]
        
        last_date = pd.to_datetime(last_date)
        print(last_date)


        if not desc:
            next_min_date = last_date + pd.Timedelta(days=1)
            params[date_key_api_min] = next_min_date.strftime('%Y-%m-%d')
        else:
            next_max_date = last_date - pd.Timedelta(days=1)
            params[date_key_api_max] = next_max_date.strftime('%Y-%m-%d')

        response = requests.get(url, params)
        if response.status_code not in [200, 206]:
            done = False
            print(f"Error: {response.status_code} - {response.text}")
            break

        if json:
            data = response.json()
            df = pd.json_normalize(data['data'])
        else:
            content = response.content.decode('utf-8')
            df = pd.read_csv(StringIO(content), sep=";")

        if len(df) == 0:
            done = True
            break

        df.to_csv(output_file, mode='a', index=False, header=False, sep=";")

    print("DONE", done)
    return


In [76]:
params = {
    "size": 20000,
    "code_departement": "75,77,78,91,92,93,94,95",
    "fields": (
        "code_station,libelle_station,code_commune,libelle_commune,latitude,"
        "longitude,code_ecoulement,libelle_ecoulement,date_observation"
    ),
    "date_observation_min": "2022-01-01",
    "date_observation_max": "2024-01-01"
}

date_key_api_min = "date_observation_min"
date_key_api_max = "date_observation_max"
date_key = "date_observation"

url = "https://hubeau.eaufrance.fr/api/v1/ecoulement/observations"
output_file = "analyses_ec.csv"

fetch_all_data(
    url,
    output_file,
    params,
    date_key_api_min,
    date_key_api_max,
    date_key,
    desc=False,
    json=True
)

DONE True


In [86]:
params = {
    "size": 20000,
    "fields": (
        "code_commune,nom_commune,date_prelevement,libelle_parametre,"
        "resultat_numerique,libelle_unite,conclusion_conformite_prelevement"
    ),
    "code_departement": "75,77,78,91,92,93,94,95",
    "date_min_prelevement": "2018-01-01",
    "date_max_prelevement": "2018-12-01",
}

date_key_api_min = "date_min_prelevement"
date_key_api_max = "date_max_prelevement"
date_key = "date_prelevement"

url = "https://hubeau.eaufrance.fr/api/v1/qualite_eau_potable/resultats_dis"

output_file = "analyses_qu.csv"

fetch_all_data(
    url,
    output_file,
    params,
    date_key_api_min,
    date_key_api_max,
    date_key,
    desc=True,
    json=True
)

Response obtained.
before while
2018-11-30 11:00:00+00:00
2018-11-28 11:30:00+00:00
2018-11-26 13:00:00+00:00
2018-11-23 13:00:00+00:00
2018-11-21 12:42:00+00:00
2018-11-19 12:29:00+00:00
2018-11-16 12:58:00+00:00
2018-11-14 11:21:00+00:00
2018-11-12 12:34:00+00:00
2018-11-09 12:13:00+00:00
2018-11-07 12:40:00+00:00
2018-11-05 13:14:00+00:00
2018-11-02 10:15:00+00:00
2018-10-31 10:14:00+00:00
2018-10-29 11:23:00+00:00
2018-10-26 14:37:00+00:00
2018-10-24 11:46:00+00:00
2018-10-22 11:52:00+00:00
2018-10-19 12:26:00+00:00
2018-10-17 11:03:00+00:00
2018-10-15 11:55:00+00:00
2018-10-12 13:30:00+00:00
2018-10-10 11:38:00+00:00
2018-10-08 12:43:00+00:00
2018-10-05 11:53:00+00:00
2018-10-03 11:50:00+00:00
2018-10-01 11:02:00+00:00
2018-09-28 15:25:00+00:00
2018-09-26 12:25:00+00:00
2018-09-24 13:18:00+00:00
2018-09-21 12:57:00+00:00
2018-09-19 12:58:00+00:00
2018-09-17 12:25:00+00:00
2018-09-14 14:15:00+00:00
2018-09-12 13:06:00+00:00
2018-09-10 12:39:00+00:00
2018-09-07 12:49:00+00:00
2018-0

In [104]:
params = {
    "size": 20000,
    "fields": (
        "code_lieusurv,libelle_lieusurv,longitude,latitude,"
        "date_prel,code_parametre,libelle_parametre,"
        "resultat_analyse,libelle_unite_resultat"
    ),
    "date_min_prel": "2021-01-01T00:00:00",
    "date_max_prel": "2024-12-31T23:59:59",
}

date_key_api_min = "date_min_prel"
date_key_api_max = "date_max_prel"
date_key = "date_prel"

url = "https://hubeau.eaufrance.fr/api/v1/surveillance_littoral/contaminants_chimiques.csv"

output_file = "analyses_co.csv"

fetch_all_data(
    url,
    output_file,
    params,
    date_key_api_min,
    date_key_api_max,
    date_key,
    desc=True,
    json=False
)

Response obtained.
before while
2021-06-24 10:20:00+00:00
2021-03-08 00:00:00+00:00
DONE True


In [87]:
df_qualite = pd.read_csv("analyses_qu.csv", sep=";")
df_qualite = pd.DataFrame(all_data)
df_qualite = df_qualite[df_qualite["libelle_unite"] != "SANS OBJET"]
df_qualite["conclusion_conformite_prelevement"] = df_qualite["conclusion_conformite_prelevement"].apply(
    lambda x: 1 if "Eau d'alimentation conforme" in x else 0
)
df_qualite.head()

,libelle_parametre,resultat_numerique,libelle_unite,code_commune,nom_commune,date_prelevement,conclusion_conformite_prelevement
0,Sodium,7.6,mg/L,92064,SAINT-CLOUD,2023-11-07T12:23:00Z,1
1,Paclobutrazole,0.0,µg/L,92064,SAINT-CLOUD,2023-11-07T12:23:00Z,1
2,"Dichloroéthane-1,2",0.0,µg/L,92064,SAINT-CLOUD,2023-11-07T12:23:00Z,1
3,Pyriméthanil,0.0,µg/L,92064,SAINT-CLOUD,2023-11-07T12:23:00Z,1
4,Thiamethoxam,0.0,µg/L,92064,SAINT-CLOUD,2023-11-07T12:23:00Z,1


In [106]:
df_ecoulement = pd.read_csv("analyses_ec.csv", sep=";")
df_littoral = pd.read_csv("analyses_co.csv", sep=";")

In [107]:
df_ecoulement.head()

,code_station,libelle_station,code_commune,libelle_commune,date_observation,code_ecoulement,libelle_ecoulement,longitude,latitude
0,F4860002,Réveillon,94070,SANTENY,2024-09-29,1,Ecoulement visible,2.573223,48.723483
1,F6640008,Morbras,94060,LA QUEUE-EN-BRIE,2024-09-29,1,Ecoulement visible,2.566385,48.787033
2,F4610002,Rue de Corbreuse,78564,SAINT-MARTIN-DE-BRETHENCOURT,2024-09-27,1,Ecoulement visible,1.927225,48.508792
3,H4070002,Etang communal,78003,ABLIS,2024-09-27,1,Ecoulement visible,1.835732,48.523302
4,H4110002,Etang guillemet,78464,ORCEMONT,2024-09-27,1,Ecoulement visible,1.807865,48.608540


In [108]:
df_littoral.head()

,code_lieusurv,libelle_lieusurv,date_prel,code_parametre,libelle_parametre,resultat_analyse,libelle_unite_resultat,longitude,latitude
0,1001104,Oye plage,2021-03-01T00:00:00Z,1652,Hexachlorobutadiène,0.07,Microgramme par Kilogramme de poids frais,2.00455,51.002767
1,1001104,Oye plage,2021-03-01T00:00:00Z,1119,Bifénox,1.00,Microgramme par Kilogramme de poids frais,2.00455,51.002767
2,1001104,Oye plage,2021-03-01T00:00:00Z,1388,Cadmium,0.48,milligramme par kilogramme de matière sèche,2.00455,51.002767
3,1001104,Oye plage,2021-03-01T00:00:00Z,1458,Anthracène,0.34,Microgramme par Kilogramme de poids frais,2.00455,51.002767
4,1001104,Oye plage,2021-03-01T00:00:00Z,1083,Chlorpyriphos-éthyl,0.20,Microgramme par Kilogramme de poids frais,2.00455,51.002767
